In [295]:
import pandas as pd
import numpy as np

#read csv file into pandas dataframe
df1 = pd.read_csv('sequencing_data_biochem1.csv') 

#replace 0's with NaN so that cycles with no reads in all 4 columns will return NaN by idxmax() function
df1.replace(to_replace={0.000: np.nan}, inplace=True)

#select columns for second cycle of biochem1 and rename columns to match format of call_1
second_cycle = df1[['A_2','C_2','G_2','T_2']].rename(columns={'A_2':'A', 'C_2':'C', 'G_2':'G', 'T_2':'T'}) 
#return column name of max value of each cycle, NaN if 0
df1['call_2']=second_cycle.idxmax(axis=1) 


In [296]:
#replace NaN in base call column with N to indicate no reads
df1['call_2'].replace(to_replace={np.nan: 'N'}, inplace=True) 
#convert NaN back to 0.000 in rest of dataframe
df1.fillna(0, inplace=True) 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df1)

    ref_1    A_1    C_1    G_1    T_1 call_1 ref_2    A_2    C_2    G_2  \
0       C  0.122  0.428  0.039  0.127      C     T  0.022  0.006  0.060   
1       T  0.155  0.138  0.102  0.680      T     C  0.186  0.390  0.107   
2       T  0.071  0.053  0.186  0.587      T     A  0.503  0.137  0.019   
3       C  0.101  0.330  0.021  0.088      C     T  0.167  0.193  0.149   
4       A  0.363  0.189  0.114  0.004      A     A  0.470  0.018  0.006   
5       G  0.005  0.099  0.506  0.110      G     T  0.119  0.003  0.081   
6       T  0.089  0.159  0.184  0.516      T     G  0.120  0.178  0.546   
7       C  0.000  0.592  0.162  0.045      C     C  0.175  0.374  0.128   
8       A  0.463  0.199  0.105  0.023      A     G  0.152  0.191  0.680   
9       G  0.141  0.060  0.692  0.115      G     A  0.100  0.092  0.026   
10      G  0.145  0.173  0.441  0.020      G     T  0.014  0.066  0.129   
11      G  0.158  0.190  0.681  0.167      G     A  0.031  0.061  0.063   
12      G  0.064  0.062  

In [297]:
#compare ref_1 to call_1 and record 1 for every mismatch or 0 if the nt readings match
df1.insert(6, 'mismatch_1', np.where(df1['ref_1'] != df1['call_1'], 1, 0)) 
#compare ref_2 to call_2 and record 1 for every mismatch or 0 if the nt readings match
df1['mismatch_2'] = np.where(df1['ref_2'] != df1['call_2'], 1, 0) 

In [298]:
#calculate error for both cycles and total error for biochem1
#done by summing 1's in mismatch column and dividing by total reads and multiplying by 100

biochem1_error1 = (sum(df1['mismatch_1']))/len(df1['ref_1'])*100 
biochem1_error2 = (sum(df1['mismatch_2']))/len(df1['ref_1'])*100
biochem1_totalerror = (sum(df1['mismatch_1']) + sum(df1['mismatch_2']))/(2*len(df1['ref_1']))*100

print(biochem1_error1, biochem1_error2, biochem1_totalerror)

3.9 6.5 5.2


In [309]:
#Biochem 2

df2 = pd.read_csv('sequencing_data_biochem2.csv')

df2.replace(to_replace = {0.000: np.nan}, inplace=True)
second_cycle1 = df2[['A_2','C_2','G_2','T_2']].rename(columns={'A_2':'A', 'C_2':'C', 'G_2':'G', 'T_2':'T'})
df2['call_2'] = second_cycle1.idxmax(axis=1)

#replace NaN with N
df2['call_2'].replace(to_replace = {np.nan: 'N'}, inplace=True)
#convert NaN back to 0.000
df2.fillna(0, inplace=True) 

#create dictionary to compensate for software error
mydict = {'C':'G', 'T':'A', 'A': 'C', 'G':'T'}

#replace values in both call columns with correct reading
df2['call_1'] = df2['call_1'].replace(mydict)
df2['call_2'] = df2['call_2'].replace(mydict)
#df2

,ref_1,A_1,C_1,G_1,T_1,call_1,ref_2,A_2,C_2,G_2,T_2,call_2
0,G,0.168,0.232,0.028,0.071,G,T,0.122,0.004,0.244,0.037,T
1,A,0.099,0.064,0.004,0.285,A,A,0.027,0.086,0.100,0.155,A
2,C,0.321,0.148,0.070,0.196,C,C,0.060,0.084,0.152,0.049,T
3,G,0.170,0.360,0.146,0.088,G,A,0.165,0.111,0.004,0.335,A
4,T,0.129,0.111,0.302,0.056,T,A,0.122,0.184,0.061,0.201,A
...,...,...,...,...,...,...,...,...,...,...,...,...
995,G,0.000,0.000,0.000,0.000,N,G,0.079,0.239,0.050,0.104,G
996,C,0.398,0.186,0.175,0.025,C,C,0.212,0.128,0.024,0.030,C
997,A,0.086,0.089,0.186,0.350,A,G,0.035,0.330,0.021,0.165,G
998,G,0.168,0.297,0.157,0.079,G,A,0.090,0.135,0.178,0.235,A


In [310]:
df2.insert(6, 'mismatch_1', np.where(df2['ref_1'] != df2['call_1'], 1, 0))
df2['mismatch_2'] = np.where(df2['ref_2'] != df2['call_2'], 1, 0)

In [311]:
#calculate error for both cycles and total error for biochem1
#done by summing 1's in mismatch column and dividing by total reads and multiplying by 100

biochem_2_error_1 = (sum(df2['mismatch_1']))/len(df2['ref_1'])*100
biochem_2_error_2 = (sum(df2['mismatch_2']))/len(df2['ref_1'])*100
biochem_2_totalerror = (sum(df2['mismatch_1']) + sum(df2['mismatch_2']))/(2*len(df2['ref_1']))*100

print(biochem_2_error_1, biochem_2_error_2, biochem_2_totalerror)

5.5 4.9 5.2


In [312]:
##Further Analysis

####Below code tests to see what percentage of mismatches were result of no reads (N)

percent_nans1 = (sum(np.where(df1['call_1'] == 'N', 1, 0)))/len(df1['ref_1'])*100
percent_nans2 = (sum(np.where(df1['call_2'] == 'N', 1, 0)))/len(df1['ref_2'])*100
percent_nans3 = (sum(np.where(df2['call_1'] == 'N', 1, 0)))/len(df2['ref_1'])*100
percent_nans4 = (sum(np.where(df2['call_2'] == 'N', 1, 0)))/len(df2['ref_2'])*100

print(percent_nans1, percent_nans2 ,percent_nans3 ,percent_nans4) #both cycles of both biochemistries return 2% N's, so this is not a factor that contributes to varying accuracies between two methods of sequencing

2.0 2.0 2.0 2.0


In [474]:
###generate summary chart for biochemistry 1
##find average signal intensity values for correct base calls

#subset the correct reads from cycle 1 of biochem 1
cycle1 = df1.iloc[:,1:6].loc[df1['mismatch_1'] == 0]
#rename columns to match those of cycle 2
cycle1.rename(columns={'A_1':'A', 'C_1':'C', 'G_1':'G', 'T_1':'T', 'call_1': 'call'}, inplace=True)

#subset the correct reads from cycle 2 of biochem 1
cycle2 = df1.iloc[:,8:13].loc[df1['mismatch_2'] == 0]
#renaame columns to match those of cycle 1
cycle2.rename(columns={'A_2':'A', 'C_2':'C', 'G_2':'G', 'T_2':'T', 'call_2': 'call'}, inplace=True)

#add the second cycle to the bottom of the first cycle
biochem1 = cycle1.append(cycle2, ignore_index=True)


#find the max signal intensity for all rows, representing the signal intensity for correct reads
biochem1['max'] = biochem1.max(axis=1)
#remove signal intensity reading columns
biochem1.drop(columns=['A', 'C', 'G', 'T'], inplace=True)
#find the mean of signal intensities for all four bases
biochem1['mean'] = biochem1.groupby(['call']).transform('mean')
#remove max signal intensity column, and remove duplicates
#only the average max intensity for each base is left
biochem1 = biochem1.drop(columns=['max']).drop_duplicates()
#sort alphabetically
biochem1.sort_values(by=['call'], inplace=True)
#rename call column to nucleotide
biochem1.rename(columns={'call':'nucleotide'}, inplace=True)


##find average signal intensity for incorrect base calls and incorrect base call counts
#subset the incorrect reads from cycle 1 of biochem 1
error1 = df1.iloc[:,0:5].loc[((df1['mismatch_1'] == 1) & (df1['call_1'] != 'N'))].reset_index(drop=True)
error1.rename(columns={'ref_1':'ref','A_1':'A', 'C_1':'C', 'G_1':'G', 'T_1':'T'}, inplace=True)

#subset the incorrect reads from cycle 2 of biochem 1
error2 = df1.iloc[:,7:12].loc[((df1['mismatch_2'] == 1) & (df1['call_2'] != 'N'))].reset_index(drop=True)
error2.rename(columns={'ref_2':'ref', 'A_2':'A', 'C_2':'C', 'G_2':'G', 'T_2':'T'}, inplace=True)

#add the second cycle to the bottom of the first cycle
biochem1_errors = error1.append(error2, ignore_index=True)

#function to return the signal intensity of the correct base call
def find_errors(df):
    ref = df.iloc[:,0]

    my_arr = []
    i = 0
    for j in ref:
        my_arr.append(df.loc[i,j])
        i += 1

    return(my_arr)

#create new column to contain signal intensity of intended base call
biochem1_errors['error'] = find_errors(biochem1_errors)
#remove signal intensity reading columns
biochem1_errors.drop(columns=['A', 'C', 'G', 'T'], inplace=True)
#find the mean of signal intensities for all four bases
biochem1_errors['mean'] = biochem1_errors.groupby(['ref']).transform('mean')
#count the number of errors for each nucleotide
biochem1_errors['error_counts'] = biochem1_errors.groupby(['ref'])['error'].transform('count')
#remove erroneous signal intensity column, and remove duplicates
#only the average max intensity for each base is left
biochem1_errors = biochem1_errors.drop(columns=['error']).drop_duplicates()
#sort alphabetically
biochem1_errors.sort_values(by=['ref'], inplace=True)
#rename call column to nucleotide
biochem1_errors.rename(columns={'ref':'nucleotide'}, inplace=True)


#merge two dataframes together to generate summary chart
summary1 = pd.merge(biochem1, biochem1_errors, on='nucleotide')
#count the total number of each nucleotides on the reference genome and append the total to summary chart
total_counts = df1['ref_1'].value_counts() + df1['ref_2'].value_counts()
summary1['total_counts'] = total_counts.values
summary1.to_csv('summary1.csv', index=False)
print(summary1)




  nucleotide    mean_x    mean_y  error_counts  total_counts
0          A  0.499485  0.066370            46           509
1          C  0.492514  0.082500             4           494
2          G  0.497853  0.052400             5           509
3          T  0.497550  0.054667             9           488


In [475]:
###generate summary chart for biochemistry 2
##find average signal intensity values for correct base calls

#subset the correct reads from cycle 1 of biochem 2
cycle3 = df2.iloc[:,1:6].loc[df2['mismatch_1'] == 0]
cycle3.rename(columns={'A_1':'A', 'C_1':'C', 'G_1':'G', 'T_1':'T', 'call_1': 'call'}, inplace=True)

#subset the correct reads from cycle 2 of biochem 2
cycle4 = df2.iloc[:,8:13].loc[df2['mismatch_2'] == 0]
cycle4.rename(columns={'A_2':'A', 'C_2':'C', 'G_2':'G', 'T_2':'T', 'call_2': 'call'}, inplace=True)

#append second cycle to bottom of first cycle
biochem2 = cycle3.append(cycle4, ignore_index=True)

#find the max signal intensity for all rows, representing the signal intensity for correct reads
biochem2['max'] = biochem2.max(axis=1)
#remove signal intensity reading columns
biochem2.drop(columns=['A', 'C', 'G', 'T'], inplace=True)
#find the mean of signal intensities for all four bases
biochem2['mean'] = biochem2.groupby(['call']).transform('mean')
#remove max signal intensity column, and remove duplicates
#only the average max intensity for each base is left
biochem2 = biochem2.drop(columns=['max']).drop_duplicates()
#sort alphabetically
biochem2.sort_values(by=['call'], inplace=True)
#rename call column to nucleotide
biochem2.rename(columns={'call':'nucleotide'}, inplace=True)


##find average signal intensity for incorrect base calls and incorrect base call counts
#subset the incorrect reads from cycle 1 of biochem 2
error3 = df2.iloc[:,0:5].loc[((df2['mismatch_1'] == 1) & (df2['call_1'] != 'N'))].reset_index(drop=True)
error3.rename(columns={'ref_1':'ref','A_1':'A', 'C_1':'C', 'G_1':'G', 'T_1':'T'}, inplace=True)

#subset the incorrect reads from cycle 2 of biochem 2
error4 = df2.iloc[:,7:12].loc[((df2['mismatch_2'] == 1) & (df2['call_2'] != 'N'))].reset_index(drop=True)
error4.rename(columns={'ref_2':'ref', 'A_2':'A', 'C_2':'C', 'G_2':'G', 'T_2':'T'}, inplace=True)

#add the second cycle to the bottom of the first cycle
biochem2_errors = error3.append(error4, ignore_index=True)

#create new column to contain signal intensity of intended base call
biochem2_errors['error'] = find_errors(biochem2_errors)
#remove signal intensity reading columns
biochem2_errors.drop(columns=['A', 'C', 'G', 'T'], inplace=True)
#find the mean of signal intensities for all four bases
biochem2_errors['mean'] = biochem2_errors.groupby(['ref']).transform('mean')
#count the number of errors for each nucleotide
biochem2_errors['error_counts'] = biochem2_errors.groupby(['ref'])['error'].transform('count')
#remove erroneous signal intensity column, and remove duplicates
#only the average max intensity for each base is left
biochem2_errors = biochem2_errors.drop(columns=['error']).drop_duplicates()
#sort alphabetically
biochem2_errors.sort_values(by=['ref'], inplace=True)
#rename call column to nucleotide
biochem2_errors.rename(columns={'ref':'nucleotide'}, inplace=True)


#merge two dataframes together to generate summary chart
summary2 = pd.merge(biochem2, biochem2_errors, on='nucleotide')
#count the total number of each nucleotides on the reference genome and append the total to summary chart
total_counts = df2['ref_1'].value_counts() + df2['ref_2'].value_counts()
summary2['total_counts'] = total_counts.values
summary2.to_csv('summary2.csv', index=False)
print(summary2)

  nucleotide    mean_x    mean_y  error_counts  total_counts
0          A  0.297766  0.104200            15           502
1          C  0.301624  0.119379            29           475
2          G  0.299207  0.110667             9           512
3          T  0.295464  0.128182            11           511
